In [13]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import duckdb
import sqlalchemy

In [14]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

c:\Users\banderson\Repos\duckdb-demo\venv\lib\site-packages\IPython\core\interactiveshell.py:3135: UserWarning: Config option `displaycon` not recognized by `SqlMagic`.  Did you mean `displaylimit`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
%sql duckdb:///:memory:

'Connected: @:memory:'

In [25]:
%%sql
SELECT VendorID, COUNT(*) as Cnt, AVG(Trip_distance) as AvgDist, SUM(tip_amount) as TotalTip
FROM read_parquet('data/yellow_tripdata_2021-01.parquet')
GROUP BY VendorID

 * duckdb:///:memory:


,VendorID,Cnt,AvgDist,TotalTip
0,1,422337,2.483790,7.922043e+05
1,2,937141,5.572211,1.835102e+06
2,6,10291,7.171585,4.407000e+01


In [33]:
%%sql
WITH T2 AS
(SELECT PULocationID, COUNT(*) as Cnt
FROM    read_parquet('data/yellow_tripdata_2021-01.parquet')
GROUP BY PULocationID
HAVING COUNT(*) >= 3
),
T3 AS 
(SELECT DOLocationID, COUNT(*) as Cnt
FROM    read_parquet('data/yellow_tripdata_2021-01.parquet')
GROUP BY DOLocationID
HAVING COUNT(*) >= 3
)
SELECT VendorID, COUNT(*) as Cnt, AVG(Trip_distance) as AvgDist, SUM(tip_amount) as TotalTip
  FROM read_parquet('data/yellow_tripdata_2021-01.parquet')    T1
  JOIN  T2 ON T1.PULocationID = T2.PULocationID
  LEFT JOIN T3 ON T1.PULocationID = T3.DOLocationID
                OR T1.DOLocationID = T3.DOLocationID
  WHERE DAY(tpep_pickup_datetime) <> 8
    AND passenger_count >= 3
    AND store_and_fwd_flag = 'N'
    and fare_amount > 0
    AND mta_tax / fare_amount > .03
    AND DATEDIFF('Minute', tpep_pickup_datetime, tpep_dropoff_datetime) > 2
  GROUP BY VendorID
  HAVING SUM(tip_amount) > 1;

 * duckdb:///:memory:


,VendorID,Cnt,AvgDist,TotalTip
0,1,15468,1.720526,24507.84
1,2,163609,1.776775,280865.86


In [ ]:
%%sql
CREATE TABLE 



In [34]:
%%sql
SELECT *
INTO test
FROM read_csv_auto('data/yellow_tripdata_2021-01.csv')
LIMIT 10;

 * duckdb:///:memory:
(duckdb.ParserException) Parser Error: SELECT INTO not supported!
[SQL: SELECT *
INTO test
FROM read_csv_auto('data/yellow_tripdata_2021-01.csv')
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [3]:
%timeit
df_pqt = pd.read_parquet('data/yellow_tripdata_2021-01.parquet')

In [4]:
%timeit
df_csv = pd.read_csv('data/yellow_tripdata_2021-01.csv')

C:\Users\banderson\AppData\Local\Temp\ipykernel_20408\3903399185.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('data/yellow_tripdata_2021-01.csv')


Lots of SQL Magic 

https://ipython.readthedocs.io/en/stable/interactive/magics.html

Data from [NYC Taxi Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)